<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/exercise_trigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [29]:
words = open('names.txt', 'r').read().splitlines()

In [27]:
import torch

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

N = torch.zeros((27, 27, 27) , dtype=torch.int32)

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2, ix3] += 1

alpha = 1
N += alpha

In [31]:
P = N.float()
P /= P.sum(2, keepdim=True)

g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix1 = 0
  ix2 = 0
  while True:
    p = P[ix1, ix2]
    ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix3])
    if ix3 == 0:
      break
    ix1, ix2 = ix2, ix3
  print(''.join(out))

junide.
ilyasid.
prelay.
ocin.
fairritoper.


In [39]:

log_likelihood = 0.0
n = 0

for w in words:
# for w in ["kanade"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1, ix2, ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    # print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-410414.9688)
nll=tensor(410414.9688)
2.092747449874878


# NN with one layer

In [56]:
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]

    xs.append([ix1, ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.shape[0]
print('number of examples: ', num)

number of examples:  196113


In [55]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True)

In [63]:
import torch.nn.functional as F

# gradient descent
for k in range(100):

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float()
  xenc = xenc.reshape(xenc.shape[0], -1)

  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)

  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -0.5* W.grad

2.267207145690918
2.267205238342285
2.2672035694122314
2.2672019004821777
2.267200231552124
2.2671985626220703
2.2671966552734375
2.267194986343384
2.267193078994751
2.2671914100646973
2.2671895027160645
2.2671878337860107
2.267186164855957
2.2671844959259033
2.2671825885772705
2.267180919647217
2.267179250717163
2.2671773433685303
2.2671756744384766
2.267174005508423
2.26717209815979
2.2671704292297363
2.2671685218811035
2.26716685295105
2.267165184020996
2.2671635150909424
2.2671618461608887
2.267160177230835
2.2671585083007812
2.2671566009521484
2.2671549320220947
2.267153024673462
2.2671515941619873
2.2671492099761963
2.2671477794647217
2.267146348953247
2.2671444416046143
2.2671425342559814
2.2671408653259277
2.267139196395874
2.267137289047241
2.2671358585357666
2.267133951187134
2.267132043838501
2.2671306133270264
2.2671287059783936
2.26712703704834
2.267125129699707
2.2671234607696533
2.2671217918395996
2.267120122909546
2.267118215560913
2.2671165466308594
2.2671148777008057


In [89]:
# sample
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
      xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
      xenc = xenc.reshape(1, -1)
      logits = xenc @ W
      counts = logits.exp()
      p = counts / counts.sum(1, keepdims=True)

      ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix3])
      if ix3 == 0:
        break
      ix1, ix2 = ix2, ix3
  print(''.join(out))

aunide.
aliasad.
ushfay.
ainn.
aui.


# Observation:
- Trigram manual approach has a better NLL score than the bigram model.
- But NN approach for the Trigram model is hard to tune below NLL score of 2.26